# Social Computing - Summer 2019
# Exercise 3: Collaborative Filtering Recommender System

Collaborative filtering is the task of automatically predicting a user's interests based on a larger user group's behavior. If a target user shares similar preferences on items or products with another user, they are likely to favor other items of that user as well. The idea of collaborative filtering recommenders is to find users the target user is similar to and recommend the items highly rated by those users. The problem of this exercise will be to implement a collaborative filtering recommender system based on the approach of [1].

[1] F. Ricci et al.: _Recommender Systems Handbook_. Springer, 2011.

## Problem 3.1: Movie Recommender System
You will work with subset of the [MovieLens](https://movielens.org) dataset which contains 1862 movies and 100k ratings. The input is provided in two data files, a listing of movies (_u.item_) and their ratings by users (_u.data_).<br>
For the _u.item_ file, each row represents a movie with its attributes separated by '|' but the only attributes that matter for the exercise are the movie ID and the movie name. For the _u.data_ file, each row represents one user's rating for a movie. The attributes are:
* user ID
* movie ID
* rating
* timestamp (not relevant for the exercise)

Your task will be to implement a collaborative filtering recommender for the given dataset.

### Collaborative Filtering Recommender
The main goal is to find a list of users similar to the target user for recommendations. There are several ways of measuring similarity, e.g. using the cosine similarity or the Euclidean distance. For simplicity reasons, the latter will be used. The estimated rating for the target user for each movie is given by

$$r_{u,i} = \frac{\sum_{v \in N_i(u)} w_{u,v}r_{v,i}}{\sum_{v \in N_i(u)} w_{u,v}}$$

where $r_{u,i}$ is the estimated recommendation of item $i$ for target user $u$. $N_i(u)$ is the set of similar users to target user $u$ for the designated item $i$. $w_{u,v}$ is the similarity score between users $u$ and $v$ (used as a weighting factor).<p>

### Task Overview
**Write a Python program that implements a simple collaborative filtering movie recommender.** The entry point to your recommendation engine should be a function that takes a user ID, and paths to the two aforementioned datasets. The function should return the top twenty recommended movies for that user (which were not rated by the user yet). The output should be a list of tuples, containing a movie and its estimated rating - sorted by the highest rating. You can either use the templates given below or design your recommendation engine the way you want, but straightforward collborative filtering is highly recommended. Make sure that the code is clean, readable, and well documented. **Test your program for user 15.**

### Task 1: Creating The Dictionaries
The first thing to do is **implementing two functions that create the dictionaries** for the movies and the user ratings. For the movies, use the movie ID as a key and the movie name as the value. Concerning the user ratings dictionary, a dict of dicts format like ```{u1_id: {m1_id: rating1, m2_id: rating 2, ...}, u2_id: {...}, ...}``` might be useful since the user's rating needs to be stored for each movie.

In [42]:
import math

def create_movie_dict(path):
    # Movie ID as key, name as value
    movie_dict = {}
    
    with open(path, encoding='utf-8') as file:        
        for line in file:
            # TODO: Extract the first two attributes, movie_id and movie_name
            first_index = line.index('|')
            second_index = line[first_index+1:].index('|') + first_index + 1
            movie_id = line[:first_index]
            movie_name = line[first_index+1:second_index]
            movie_dict[movie_id] = movie_name
    
    return movie_dict

def create_user_rating_dict(path):
    # User ID as key, dict of ratings as values
    user_rating_dict = {}
    
    with open(path, encoding='utf-8') as file:        
        for line in file:
            # TODO: Extract all the information except for timestamp
            first_index = line.index("\t")
            second_index = line[first_index+1:].index("\t") + first_index + 1
            third_index = line[second_index+1:].index("\t") + second_index + 1
            user_id = line[:first_index]
            movie_id = line[first_index+1:second_index]
            rating = line[second_index+1:third_index]
            
            if user_id in user_rating_dict:
                movie_rating_dict={}
                movie_rating_dict = user_rating_dict[user_id]
                movie_rating_dict[movie_id]=rating
                user_rating_dict[user_id]=movie_rating_dict
            else:
                movie_rating_dict={}
                movie_rating_dict[movie_id]=rating
                user_rating_dict[user_id]=movie_rating_dict
    
    return user_rating_dict

### Task 2: Calculating Similarity Scores
The task is to **implement a function that calculates the similarity score $w_{u,v}$** between two users $u$ and $v$. This should be done for all users and the movies $i$ they both rated. For the similarity score, use the _normalized_ Euclidean distance measure given by
$$ w_{u,v} = \frac{1}{1 + \sum_{i} (r_{u,i} - r_{v,i})^2} $$

In [43]:
# Calculates simularity using Euclidean distance 
def calculate_similarity_score(ratings, u1_id, u2_id):
    common_movies = []
    
    # TODO: Find common ratings between the two users
    movie_ratings_u1 = ratings[u1_id]
    movie_ratings_u2 = ratings[u2_id]
    common_movies = movie_ratings_u1.keys() & movie_ratings_u2.keys()
    
    # No common ratings -> similarity is 0
    if len(common_movies) == 0:
        return 0

    # TODO: Calculate normalized Euclidean distance between two users based on their common ratings
    w = 0.0    
    for m_id in common_movies:
        # TODO: Implement the above formula
        w = w + (float(movie_ratings_u1[m_id]) - float(movie_ratings_u2[m_id])) ** 2
    
    w = 1 / (1 + w)
        
    return w

### Task 3: The Recommender System
Now it is time to **implement the collabortive filtering recommender system that outputs the recommended movies for a target user**. Use the functionality defined above to compute the similarity scores between them and calculate the resulting estimated ratings. Sort the movies according to the highest rating before returning them.

In [44]:
def cf_recommend(ratings, tgt_u_id):
    w_ratings = {} # Weighted ratings
    sim_scores = {} # Similarity scores
    recommended_movies = [] # Each element is a tuple (est_rating, m_id)
    
    for u_id in ratings:
        # Compute similarity scores between the target user and all others
        if u_id != tgt_u_id:
            w = calculate_similarity_score(ratings, tgt_u_id, u_id)
            
            # If 0, estimated recommendation will be 0
            if w > 0:
                # Check movies rated by user (but not target user)
                for m_id in ratings[u_id]:                    
                    if m_id not in ratings[tgt_u_id]:
                        # TODO: Calculate weighted ratings and add them up
                        if m_id not in w_ratings:
                            w_ratings[m_id] = 0.0
                        w_ratings[m_id] += w * float(ratings[u_id][m_id])
                        
                        # TODO: Add up similarity scores
                        if m_id not in sim_scores:
                            sim_scores[m_id] = 0.0
                        sim_scores[m_id] += w

    for m_id in w_ratings: # TODO: For each movie
        est_rating = w_ratings[m_id] / sim_scores[m_id]  # Estimate the rating
        recommended_movies.append((est_rating, m_id))
        
    # TODO: Sort the list
    recommended_movies.sort(key=lambda x: x[0])
    recommended_movies.reverse()
    
    return recommended_movies


# Create dictionaries
movie_dict = create_movie_dict('movielens/u.item')
ratings_dict = create_user_rating_dict('movielens/u.data')

# TODO: Run for user 15
target_user_id = '15'
recommended_movies = cf_recommend(ratings_dict, target_user_id)  # Call the function

# Show the top twenty entries
top_twenty = []
for est_rating, m_id in recommended_movies[:20]:
    top_twenty.append((movie_dict[m_id], est_rating))

print('Recommendation for user', target_user_id, ':', top_twenty)

Recommendation for user 15 : [('Prefontaine (1997)', 5.000000000000001), ('Saint of Fort Washington, The (1993)', 5.000000000000001), ('Entertaining Angels: The Dorothy Day Story (1996)', 5.0), ("Someone Else's America (1995)", 5.0), ('Aiqing wansui (1994)', 5.0), ('Star Kid (1997)', 5.0), ('Marlene Dietrich: Shadow and Light (1996) ', 5.0), ('Santa with Muscles (1996)', 5.0), ('They Made Me a Criminal (1939)', 5.0), ('Great Day in Harlem, A (1994)', 5.0), ('Letter From Death Row, A (1998)', 4.886561954624781), ('Bitter Sugar (Azucar Amargo) (1996)', 4.853953971275307), ('Pather Panchali (1955)', 4.824191713371869), ('World of Apu, The (Apur Sansar) (1959)', 4.7578639291960725), ("C'est arrivé près de chez vous (1992)", 4.677186255714496), ("Some Mother's Son (1996)", 4.665094339622642), ('Braindead (1992)', 4.648287265011339), ('Close Shave, A (1995)', 4.604920391555617), ('Paths of Glory (1957)', 4.585030356242251), ('Faust (1994)', 4.584110922709905)]
